In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import Fragments
from ClickReaction import BocRemoval
from ClickReaction import CuAAC
from rdkit.Chem import  Descriptors
from rdkit.Chem.Draw import IPythonConsole
import io
import itertools
from PIL import Image
from ClickReaction import BocRemoval
from tqdm import tqdm
import re
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen
import json
import csv
import numpy as np

In [2]:
def replace_specific_pattern(text, pattern, replacement):
    return re.sub(pattern, replacement, text)
rxn = Chem.ReactionFromSmarts('[*:6][C:1]#[C:2].[N-:3]=[N+:4]=[N:5]>>[N:5]1([C:2]=[C:1]([*:6])[N:3]=[N:4]1)')

In [3]:
def convert_primary_amines_to_diazonium(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    primary_amine_pattern = Chem.MolFromSmarts('[NX3;H2]')  
    diazonium_pattern = Chem.MolFromSmiles('[N]=[N+]=[N-]')
    wrong_smiles = Chem.MolFromSmiles('C([N]=[N+]=[N-])[N]=[N+]=[N-]')
    wrong_smiles_2 = Chem.MolFromSmiles('C(=O)[N]=[N+]=[N-]')
    true_smiles = Chem.MolFromSmiles('C(N)N')
    true_smiles_2 = Chem.MolFromSmiles('C(=O)N')
    if molecule.HasSubstructMatch(primary_amine_pattern):
        modified_molecule = Chem.ReplaceSubstructs(molecule, primary_amine_pattern, diazonium_pattern, replaceAll=True)[0]
        if modified_molecule.HasSubstructMatch(wrong_smiles):
            modified_molecule = Chem.ReplaceSubstructs(modified_molecule, wrong_smiles, true_smiles, replaceAll=True)[0]
        elif modified_molecule.HasSubstructMatch(wrong_smiles_2):
            modified_molecule = Chem.ReplaceSubstructs(modified_molecule, wrong_smiles_2, true_smiles_2, replaceAll=True)[0]
        new_smiles = Chem.MolToSmiles(modified_molecule, isomericSmiles=True)
        return new_smiles
    elif '[NH2+]' in smiles:
        try:
            boc_protected_amine = Chem.MolFromSmiles(smiles)
            reaction = BocRemoval(bocamine=boc_protected_amine)
            molecule = reaction.get_product()
            modified_molecule = Chem.ReplaceSubstructs(molecule, primary_amine_pattern, diazonium_pattern, replaceAll=True)[0]
            new_smiles = Chem.MolToSmiles(modified_molecule, isomericSmiles=True)
            return new_smiles
        except:
            # print(smiles,"无法脱Boc")
            return None
    else:
        # print( smiles,"未找到一级胺")
        return None